In [2]:
import os
import sys
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Activation, Conv2D, Input, Embedding, Reshape, MaxPool2D, Concatenate, Flatten, Dropout, Dense, Conv1D
from keras.layers import MaxPool1D, LSTM, Bidirectional
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
import gensim

In [3]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

In [4]:
import pandas as pd
df = pd.read_csv("./data.csv")

In [5]:
df.dropna(subset=['Body'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [6]:
df['Headline'] = df['Headline'].str.lower()
df['Body'] = df['Body'].str.lower()
df.head()

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-414191...,four ways bob corker skewered donald trump,image copyright getty images\non sunday mornin...,1
1,https://www.reuters.com/article/us-filmfestiva...,linklater's war veteran comedy speaks to moder...,"london (reuters) - “last flag flying”, a comed...",1
2,https://www.nytimes.com/2017/10/09/us/politics...,trump’s fight with corker jeopardizes his legi...,the feud broke into public view last week when...,1
3,https://www.reuters.com/article/us-mexico-oil-...,egypt's cheiron wins tie-up with pemex for mex...,mexico city (reuters) - egypt’s cheiron holdin...,1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,jason aldean opens 'snl' with vegas tribute,"country singer jason aldean, who was performin...",1


In [7]:
import string
def remove_punctuations(text):
    punctuations = set(string.punctuation)
    text = str(text)
    # return text.translate(str.maketrans('', '', punctuations))
    return " ".join([word for word in text.split() if word not in punctuations])

df['Headline'] = df['Headline'].apply(lambda x: remove_punctuations(x))
df['Body'] = df['Body'].apply(lambda x: remove_punctuations(x))

In [8]:
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in text.split() if word not in STOPWORDS])

df['Headline'] = df['Headline'].apply(lambda x: remove_stopwords(x))
df['Body'] = df['Body'].apply(lambda x: remove_stopwords(x))

In [9]:
import re
def remove_spl_chars(text):
    text = re.sub('[^a-zA-Z0-9]', ' ', text)
    text = re.sub('\s+', ' ', text)
    return text

df['Headline'] = df['Headline'].apply(lambda x: remove_spl_chars(x))
df['Body'] = df['Body'].apply(lambda x: remove_spl_chars(x))

In [10]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer(language='english')
def stem(text):
    stemmed_sentence = " ".join(stemmer.stem(word) for word in text.split())
    return stemmed_sentence

df['Headline'] = df['Headline'].apply(lambda x: stem(x))
df['Body'] = df['Body'].apply(lambda x: stem(x))

In [11]:
texts = []
labels = []
for i in range(3988):
    texts.append(df['Body'][i])
    labels.append(df['Label'][i])

print(len(texts))
len(labels)

3988


3988

In [12]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [13]:
def prep_vect(text):
    doc = nlp(text)

    filtered_tokens = []
    for token in doc:
        if token.is_punct or token.is_stop:
            continue
        filtered_tokens.append(token.lemma_)
    
    return wv.get_mean_vector(filtered_tokens)

In [14]:
df['vector'] = df['Body'].apply(lambda text: prep_vect(text))

In [15]:
df['vector'][0].shape

(300,)

In [16]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df.vector.values, df.Label, test_size=0.2, random_state=2022, stratify=df.Label)

In [17]:
x_train.shape

(3190,)

In [18]:
# Here x_train is not a native 2D numpy array but a numpy array within a numpy array
x_train_2d = np.stack(x_train) 
x_test_2d = np.stack(x_test) 

In [19]:
x_train_2d.shape

(3190, 300)

In [20]:
def RNN():
    inputs = Input(name='inputs',shape=[300])
    # layer = Embedding(max_words,100,input_length=300)(inputs)
    layer = Reshape((1, 300))(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [21]:
from keras.optimizers import RMSprop

model_lstm = RNN()
model_lstm.summary()
model_lstm.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 300)]             0         
                                                                 
 reshape (Reshape)           (None, 1, 300)            0         
                                                                 
 lstm (LSTM)                 (None, 64)                93440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

In [72]:
# In the above model in the shape of input for LSTM (batch_size, time_step, number of features)

In [73]:
model_lstm.fit(x_train_2d,y_train, batch_size=128, epochs=20, validation_split=0.1)

Epoch 1/20


23/23 [==============================] - 2s 27ms/step - loss: 0.6905 - accuracy: 0.5301 - val_loss: 0.6946 - val_accuracy: 0.4828
Epoch 2/20
23/23 [==============================] - 0s 6ms/step - loss: 0.6876 - accuracy: 0.5371 - val_loss: 0.6954 - val_accuracy: 0.4828
Epoch 3/20
23/23 [==============================] - 0s 6ms/step - loss: 0.6843 - accuracy: 0.5371 - val_loss: 0.6874 - val_accuracy: 0.4828
Epoch 4/20
23/23 [==============================] - 0s 8ms/step - loss: 0.6773 - accuracy: 0.5388 - val_loss: 0.6804 - val_accuracy: 0.4828
Epoch 5/20
23/23 [==============================] - 0s 6ms/step - loss: 0.6614 - accuracy: 0.5831 - val_loss: 0.6563 - val_accuracy: 0.6583
Epoch 6/20
23/23 [==============================] - 0s 6ms/step - loss: 0.6322 - accuracy: 0.7102 - val_loss: 0.6179 - val_accuracy: 0.7461
Epoch 7/20
23/23 [==============================] - 0s 6ms/step - loss: 0.5844 - accuracy: 0.7753 - val_loss: 0.5614 - val_accuracy: 0.7868
Epoch 8/20
23/23 [============

In [74]:
y_pred = model_lstm.predict(x_test_2d)

25/25 [==============================] - 0s 2ms/step


In [76]:
pred = []
for p in y_pred:
    if p[0] >= 0.5:
        pred.append(1)
    else:
        pred.append(0)